In [1]:
import nltk
# Download both old and new tokenizer data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Bag of Words

In [2]:
import nltk, string, pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    "This phone has great battery life",
    "Battery life on this phone is poor",
    "I love the camera on this phone"
]

# download once per session
nltk.download('punkt'); nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    return [w for w in word_tokenize(text) if w not in stop_words]

tokens_list = [preprocess(doc) for doc in corpus]
vocab = sorted({w for sent in tokens_list for w in sent})

def bow_vector(tokens):
    return [tokens.count(term) for term in vocab]

manual_bow = [bow_vector(t) for t in tokens_list]
manual_df  = pd.DataFrame(manual_bow, columns=vocab)

cv = CountVectorizer(lowercase=True, stop_words='english')
cv_bow = cv.fit_transform(corpus).toarray()
cv_df  = pd.DataFrame(cv_bow, columns=cv.get_feature_names_out())

print(manual_df, '\n'); print(cv_df)

   battery  camera  great  life  love  phone  poor
0        1       0      1     1     0      1     0
1        1       0      0     1     0      1     1
2        0       1      0     0     1      1     0 

   battery  camera  great  life  love  phone  poor
0        1       0      1     1     0      1     0
1        1       0      0     1     0      1     1
2        0       1      0     0     1      1     0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### N-grams Generation and Analysis

In [3]:
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
sentence = "Natural language processing is fascinating and powerful"
tokens   = word_tokenize(sentence.lower())

for n in range(1, 5):
    grams = list(ngrams(tokens, n))
    print(f"{n}-grams:", grams)

bigram_freq  = Counter(ngrams(tokens, 2))
trigram_freq = Counter(ngrams(tokens, 3))
print("Top bigrams:", bigram_freq.most_common())
print("Top trigrams:", trigram_freq.most_common())

1-grams: [('natural',), ('language',), ('processing',), ('is',), ('fascinating',), ('and',), ('powerful',)]
2-grams: [('natural', 'language'), ('language', 'processing'), ('processing', 'is'), ('is', 'fascinating'), ('fascinating', 'and'), ('and', 'powerful')]
3-grams: [('natural', 'language', 'processing'), ('language', 'processing', 'is'), ('processing', 'is', 'fascinating'), ('is', 'fascinating', 'and'), ('fascinating', 'and', 'powerful')]
4-grams: [('natural', 'language', 'processing', 'is'), ('language', 'processing', 'is', 'fascinating'), ('processing', 'is', 'fascinating', 'and'), ('is', 'fascinating', 'and', 'powerful')]
Top bigrams: [(('natural', 'language'), 1), (('language', 'processing'), 1), (('processing', 'is'), 1), (('is', 'fascinating'), 1), (('fascinating', 'and'), 1), (('and', 'powerful'), 1)]
Top trigrams: [(('natural', 'language', 'processing'), 1), (('language', 'processing', 'is'), 1), (('processing', 'is', 'fascinating'), 1), (('is', 'fascinating', 'and'), 1), (

### TF-IDF Vectorization of News Headlines

In [4]:
from math import log
from sklearn.feature_extraction.text import TfidfVectorizer
docs = [
    "Stock market crashes amid global uncertainty",
    "Global leaders discuss climate change solutions"
]

# scikit-learn path
vec = TfidfVectorizer(lowercase=True, stop_words='english')
tfidf = vec.fit_transform(docs).toarray()
print(pd.DataFrame(tfidf, columns=vec.get_feature_names_out()))

# manual IDF (for pedagogy)
tokens = [word_tokenize(d.lower()) for d in docs]
stop = set(stopwords.words('english'))
proc   = [[w for w in t if w.isalpha() and w not in stop] for t in tokens]
vocab  = sorted({w for doc in proc for w in doc})
idf    = {t: log(len(docs) / sum(t in p for p in proc)) for t in vocab}
tfidf_manual = []
for doc in proc:
    length = len(doc)
    tfidf_manual.append([doc.count(t)/length*idf[t] for t in vocab])
print(pd.DataFrame(tfidf_manual, columns=vocab))

      amid   change  climate  crashes  discuss    global  leaders   market  \
0  0.42616  0.00000  0.00000  0.42616  0.00000  0.303216  0.00000  0.42616   
1  0.00000  0.42616  0.42616  0.00000  0.42616  0.303216  0.42616  0.00000   

   solutions    stock  uncertainty  
0    0.00000  0.42616      0.42616  
1    0.42616  0.00000      0.00000  
       amid    change   climate   crashes   discuss  global   leaders  \
0  0.115525  0.000000  0.000000  0.115525  0.000000     0.0  0.000000   
1  0.000000  0.115525  0.115525  0.000000  0.115525     0.0  0.115525   

     market  solutions     stock  uncertainty  
0  0.115525   0.000000  0.115525     0.115525  
1  0.000000   0.115525  0.000000     0.000000  
